# Projet python 2A : évaluation de la valeure des footballeurs

César Fabiani - Mathis Lecoq

_Ce projet est réalisé dans le cadre du cours de Python de Lino Galiana pour l'année 2023-2024._

### Introduction

**Notre objectif** : Evaluer la valeur d'un joueur. Pour cela nous avons utilisé les données sur le site [**FB**REF](https://fbref.com/fr/) qui contiennent des données détaillés (buts, passes, etc) pour chaque joueur dans chaque match.

**Le projet** : Notre projet se décompose en N étapes : 
1. Extraction des données en scrappant le site [**FB**REF](https://fbref.com/fr/)
2. Identification des variables d'intérets
3. ...

### Structure du répertoire

À la racine du répertoire, on trouve :
- **`README.md`** : présentation détaillée du projet

### Installations et importations

La partie scrapping prenant beaucoup de temps pour éviter de se faire bloquer par le site, elle n'est pas incluse dans ce notebook. Vous pouvez la trouver dans le fichier **`scrapping`**.
Pour information, les installations et importations sont les suivantes :

In [7]:
!sudo apt-get install python3-pip python-dev
!pip install lxml

import bs4
import lxml
import urllib

from urllib import request

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package python3-pip
E: Unable to locate package python-dev


De plus, on utilise le module **`time`** pour espacer les requêtes et le module **`json`** pour stocker les données recueillies : 

In [8]:
import json
import time

Pour traiter les données, nous avons utilisé le module **`pandas`** :

In [9]:
import pandas as pd

### Étape 1 : Extraction des données en scrappant le site [**FB**REF](https://fbref.com/fr/)

_Le code de la partie scraping ne se trouve pas dans le notebook car prennant trop de temps pour s'éxecuter. Vous pouvez le trouvez dans le dossier **`scraping`**.

La partie scraping s'est déroulée en deux étapes. Premièrement le programme **`scraping_FBREF_1`** permet pour une ligue donnée et une saison donnée d'avoir l'ensemble des url des rapports de match correspondant à cette ligue et à cette saison. La liste des url des matchs est exportée vers le fichier json **`liste_url.json`**. Ensuite, les url récoltées sont utilisées par le programme **`scrap_ligue`** qui vient scrapper les tableaux rassemblant les données de chaque joueur. Ainsi une ligne de ce tableau correspond à la performence d'un joueur lors d'un match. Puis les données sont exportées vers le fichier **`statistiques`**. Voici pour exemple les premières lignes du fichier : 

In [13]:
path = "data/data_premierleague_2022_2023_V1.csv"
df = pd.read_csv(path)
print(df.head())

   Unnamed: 0                Joueur  Numéro de maillot   Nation Emplacement  \
0           0       Odsonne Édouard                 22   fr FRA          FW   
1           1  Jean-Philippe Mateta                 14   fr FRA          FW   
2           2         Wilfried Zaha                 11   ci CIV          LW   
3           3           Jordan Ayew                  9   gh GHA       RW,AM   
4           4          Eberechi Eze                 10  eng ENG          AM   

      Âge  Minutes  Buts  Passes décisives_x  Pénaltys marqués  ...  \
0  24-201       57     0                   0                 0  ...   
1  25-038       33     0                   0                 0  ...   
2  29-268       90     0                   0                 0  ...   
3  30-328       90     0                   0                 0  ...   
4  24-037       85     0                   0                 0  ...   

   Centres_y  Interceptions  Tacles réussis_y  Pénaltys réussis  \
0          0              0    

Une fois la base de données obtenue, nous avons nettoyé la base afin d'obtenir des données sur lesquelles nous pouvons travailler. 